### Import Libraries

In [1]:
import warnings
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn import tree
from sklearn import metrics
from joblib import dump,load
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
# !mkdir Pretrained

In [3]:
#Load data
# data_path = '/kaggle/input/cancerdataset/METABRIC_RNA_Mutation.csv'
data_path = './data/METABRIC_RNA_Mutation.csv'
master_data = pd.read_csv(data_path,low_memory=False)
master_data = master_data.drop(['patient_id'],axis=1)
master_data = master_data.drop(['death_from_cancer'],axis=1)
master_data = master_data.dropna()

#Split Features and Labels
# y = master_data['overall_survival'].values
# data = master_data.drop(['overall_survival'], axis=1)

#Split Numetical and Categorical Features
# numericalFeats = data.select_dtypes('number').columns
# categoricalFeats = data.select_dtypes('object').columns
# catdata = data[categoricalFeats]
# numdata = data[numericalFeats]

# Scale Numerical Table
# numdata_scaled = preprocessing.scale(numdata)

# One-hot/Label Encode Categorical Table
# encoder = OneHotEncoder(handle_unknown='ignore')
# encoder = LabelEncoder()
# encoder.fit(catdata)
# dump(encoder, './Pretrained/One_Hot_Encoder.joblib')
# dump(encoder, './Pretrained/Label_Encoder.joblib')
# catendata = encoder.transform(catdata).toarray()

# Combine Features
# X = np.concatenate((numdata_scaled, catendata), axis=1)

# Separate Train and Testing Data
# (X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.30, random_state=42)

### Model Training

In [4]:
# def model_training():
    
#     print('Working on Logistic Regression Model')
#     clf_LR = LogisticRegression()
#     lr_params = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
#     clf_LR = GridSearchCV(clf_LR, lr_params, cv=10, scoring='roc_auc',refit = True)
#     clf_LR.fit(X_train, y_train)
#     dump(clf_LR, './Pretrained/Logistic_Regression.joblib')
    
#     print('Working on Decision Tree Model')
#     clf_DT = tree.DecisionTreeClassifier()
#     dt_params = {'ccp_alpha': [0.1, .01, .001],'max_depth' : [5, 6, 7, 8, 9]}
#     clf_DT = GridSearchCV(clf_DT, dt_params, cv=10, scoring='roc_auc',refit = True)
#     clf_DT.fit(X_train, y_train)
#     dump(clf_DT, './Pretrained/Decision_Tree.joblib')
    
#     print('Working on Support Vector Model')
#     clf_SVM = svm.SVC()
#     svm_params = {'C': [0.1, 1, 10, 100],'gamma': [1, 0.1, 0.01, 0.001]} 
#     clf_SVM = GridSearchCV(clf_SVM, svm_params, cv=10, scoring='roc_auc',refit = True)
#     clf_SVM.fit(X_train, y_train)
#     dump(clf_SVM, './Pretrained/Support_Vector_Machine.joblib')
    
#     print('Working on Random Forest Model')
#     clf_RF = RandomForestClassifier()
#     rf_params = {'max_depth': list(range(10, 18)),\
#              'max_features': list(range(0,14)),\
#              'n_estimators':[50, 100, 120]}
#     clf_RF = GridSearchCV(clf_RF, rf_params, cv = 10, scoring='roc_auc',refit = True)
#     clf_RF.fit(X_train, y_train)
#     dump(clf_RF, './Pretrained/Random_Forest.joblib')
     
#     print('Working on XGBosst Model')
#     clf_XGB = XGBClassifier()
#     xgb_params = {'gamma': [0.5, 1], 'max_depth': [2, 3, 5, 8]}
#     clf_XGB = GridSearchCV(clf_XGB, xgb_params, cv = 10, scoring='roc_auc',refit = True)
#     clf_XGB.fit(X_train, y_train)
#     dump(clf_XGB, './Pretrained/XGBoost.joblib')

### Function to Compute Performance Metrics

In [15]:
def performanceResults(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(len(y_hat)):
        if y_actual[i] == y_hat[i] == 1:
            TP += 1
        if y_hat[i] == 1 and y_actual[i] != y_hat[i]:
            FP += 1
        if y_actual[i] == y_hat[i] == 0:
            TN += 1
        if y_hat[i] == 0 and y_actual[i] != y_hat[i]:
            FN += 1
    acc = (TP + TN) / (TP + FP + TN + FN)
    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    fpr = FP / (FP + TN)
    return acc, sensitivity, specificity, fpr

In [16]:
def final_fun_1(X_raw, best_model_path):
    #Load pretrained models
    clf = load(best_model_path)
#     onehot_path = '/kaggle/working/Pretrained/One_Hot_Encoder.joblib'
    onehot_path = './Pretrained/One_Hot_Encoder.joblib'
    encoder = load(onehot_path)
    
    #Split Numetical and Categorical Features
    numericalFeats = X_raw.select_dtypes('number').columns
    categoricalFeats = X_raw.select_dtypes('object').columns
    catdata = X_raw[categoricalFeats]
    numdata = X_raw[numericalFeats]

    # Scale Numerical Table
    numdata_scaled = preprocessing.scale(numdata)

    # One-hot Encode Test data
    catendata = encoder.transform(catdata).toarray()

    # Combine Features
    X_test = np.concatenate((numdata_scaled, catendata), axis=1)
    
    y_pred= clf.predict(X_test)
    return y_pred

In [26]:
def final_fun_2(X_raw,y_raw):
#     best_model_path =  '/kaggle/input/metabric-pretrained/Pretrained/XGBoost.joblib'
    best_model_path =  './Pretrained/Random_Forest.joblib'
    y_pred = final_fun_1(X_raw, best_model_path)
    print(y_pred)
    PR = performanceResults(y_raw, y_pred)
    (lrfpr, lrtpr, lrthresholds) = metrics.roc_curve(y_raw, y_pred)
    roc_auc = metrics.auc(lrfpr, lrtpr)
    return PR, roc_auc

In [27]:
# model_training()
raw_data = master_data.sample(n=4)

In [28]:
raw_data.head()

,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status_measured_by_ihc,er_status,...,mtap_mut,ppp2cb_mut,smarcd1_mut,nras_mut,ndfip1_mut,hras_mut,prps2_mut,smarcb1_mut,stmn2_mut,siah1_mut
1405,69.56,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0,LumB,3.0,Positve,Positive,...,0,0,0,0,0,0,0,0,0,0
1270,67.25,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumB,3.0,Positve,Positive,...,0,0,0,0,0,0,0,0,0,0
1105,62.68,BREAST CONSERVING,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,0,LumA,3.0,Positve,Positive,...,0,0,0,0,0,0,0,0,0,0
1048,73.03,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0,LumA,3.0,Positve,Positive,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# raw_data['overall_survival'].head()

In [30]:
#Split Features and Labels
raw_y_values = raw_data['overall_survival'].values
raw_feature_values = raw_data.drop(['overall_survival'], axis=1)

In [31]:
# raw_feature_values.head()

In [32]:
# raw_feature_values.to_csv('test_input.csv', index=False)

In [33]:
PR, roc_auc = final_fun_2(raw_feature_values, raw_y_values)

[1 0 0 0]


In [34]:
print('Best Model Performance Metrics on Test Input Data')
print ('Accuracy: ', PR[0])
print ('Sensitivity: ', PR[1])
print ('Specificity: ', PR[2])
print ('False Positive Rate: ', PR[3])
print('Area Under the Curve (AUC):', roc_auc)

Best Model Performance Metrics on Test Input Data
Accuracy:  0.75
Sensitivity:  0.5
Specificity:  1.0
False Positive Rate:  0.0
Area Under the Curve (AUC): 0.75


In [ ]:
# pip freeze